In [44]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.optim.lr_scheduler import CosineAnnealingLR

In [45]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')

In [91]:
import pandas as pd
import glob
import numpy as np
from sklearn.preprocessing import RobustScaler
import torch

# CSV 파일 경로 리스트 (001.csv부터 101.csv까지)
squat_file_paths = glob.glob('squat_csv_data/*.csv')  # 'path_to_files/'는 파일들이 저장된 디렉토리 경로입니다.

# 슬라이딩 윈도우 크기
window_size = 10
stride = 1

# 입력 데이터와 출력 데이터 저장 리스트
squat_X_data = []
squat_y_data = []

# 각 파일을 처리
for file_path in squat_file_paths:
    # 파일 읽기
    df = pd.read_csv(file_path, header=None, names=['frame', 'node_id', 'x', 'y', 'z'])
    
    # node_id 별로 데이터 그룹화
    for node_id in df['node_id'].unique():
        node_data = df[df['node_id'] == node_id].set_index('frame')[['x', 'y', 'z']]

        # 슬라이딩 윈도우 적용
        for start in range(0, len(node_data) - window_size):
            end = start + window_size
            # 입력값 (슬라이딩 윈도우)
            window_input = node_data.iloc[start:end].values.astype('float64')

            # 윈도우 별로 정규화
            scaler = RobustScaler()
            scaler_fitted =  scaler.fit(window_input)
            window_input_normalized = scaler_fitted.transform(window_input)  # 각 윈도우별로 정규화
            
            # 출력값 (다음 시점의 (x, y, z) 값)
            next_point = node_data.iloc[end].astype('float64')

            squat_X_data.append(window_input_normalized)
            squat_y_data.append(next_point.values)
        
# X_data는 입력값 (슬라이딩 윈도우)
# y_data는 출력값 (다음 시점의 (x, y, z))

# numpy 배열로 변환
squat_X_data = torch.FloatTensor(np.array(squat_X_data))
squat_y_data = torch.FloatTensor(np.array(squat_y_data))

print("X_data shape:", squat_X_data.shape)
print("y_data shape:", squat_y_data.shape)

# 결과 예시 (슬라이딩 윈도우의 첫 번째와 두 번째 샘플)
print("Sample input (X):", squat_X_data[0])
print("Sample output (y):", squat_y_data[0])


X_data shape: torch.Size([37050, 10, 3])
y_data shape: torch.Size([37050, 3])
Sample input (X): tensor([[-0.6779, -1.7277,  0.9513],
        [-0.6523, -1.1336,  0.8668],
        [-0.5511, -0.7513,  0.2365],
        [-0.4703, -0.3556,  0.0376],
        [-0.1769, -0.1026, -0.0376],
        [ 0.1769,  0.1026, -0.1189],
        [ 0.4091,  0.2420, -1.0924],
        [ 0.4890,  0.3829, -1.2436],
        [ 0.9050,  0.5006, -1.0335],
        [ 1.4893,  0.5572,  0.0708]])
Sample output (y): tensor([0.5171, 0.5103, 0.0047])


In [95]:
import torch
import torch.nn as nn

class LSTMmodel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMmodel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # LSTM 레이어
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size ,num_layers=2, batch_first=True)
        
        # 출력 레이어
        self.linear = nn.Sequential(
            nn.Linear(self.hidden_size, 3),
            nn.Tanh()
        )
    
    def forward(self, x):
        # LSTM 레이어 통과
        lstm_out, (h_n, c_n) = self.lstm(x)  # lstm_out은 모든 시점의 출력
        
        # 마지막 시점의 출력만 사용
        last_hidden_state = lstm_out[:, -1, :]  # (batch_size, hidden_size)
        
        # 출력 레이어 통과
        output = self.linear(last_hidden_state)
        return output


In [96]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
squat_train = TensorDataset(squat_X_data,squat_y_data)
squat_dataloader = DataLoader(squat_train, batch_size = 256, shuffle = False)


In [97]:

model = LSTMmodel(3,10).to(device)
def saveModel():
    torch.save(model.state_dict(), f'model/squat_lstm_train.pt')
criterion = nn.MSELoss()
optimizer = torch.optim.RAdam(model.parameters(), lr=1e-2)
scheduler = CosineAnnealingLR(optimizer,T_max= 25, eta_min=1e-4)
# 학습 루프
epochs = 100
min_loss = 1e+10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for x, y in squat_dataloader:
        input = x.to(device)        
        optimizer.zero_grad()

        # 순전파
        output = model(input)

        # 손실 계산
        loss = criterion(output,y.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    mean_loss =running_loss / len(squat_dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(squat_dataloader)}")
    if min(mean_loss,min_loss) == mean_loss:
        min_loss = mean_loss
        print('model_saved')
        saveModel()
    

Epoch [1/100], Loss: 0.043926749005913736
model_saved
Epoch [2/100], Loss: 0.01201409312639514
model_saved
Epoch [3/100], Loss: 0.011989818150498743
model_saved
Epoch [4/100], Loss: 0.011908176849628317
model_saved
Epoch [5/100], Loss: 0.011765012275106434
model_saved
Epoch [6/100], Loss: 0.011589674826647188
model_saved
Epoch [7/100], Loss: 0.01144298113850427
model_saved
Epoch [8/100], Loss: 0.011316695326040018
model_saved
Epoch [9/100], Loss: 0.011208913528264083
model_saved
Epoch [10/100], Loss: 0.01111174694037643
model_saved
Epoch [11/100], Loss: 0.011021128746456114
model_saved
Epoch [12/100], Loss: 0.010934824763058588
model_saved
Epoch [13/100], Loss: 0.010855206275551484
model_saved
Epoch [14/100], Loss: 0.010782366424220903
model_saved
Epoch [15/100], Loss: 0.010710760353711145
model_saved
Epoch [16/100], Loss: 0.010644454176634037
model_saved
Epoch [17/100], Loss: 0.010585242901639692
model_saved
Epoch [18/100], Loss: 0.010531430754104051
model_saved
Epoch [19/100], Loss: 

In [98]:
import pandas as pd
import glob
import numpy as np
from sklearn.preprocessing import RobustScaler
import torch

# CSV 파일 경로 리스트 (001.csv부터 101.csv까지)
push_file_paths = glob.glob('push_csv_data/*.csv')  # 'path_to_files/'는 파일들이 저장된 디렉토리 경로입니다.

# 슬라이딩 윈도우 크기
window_size = 10
stride = 1

# 입력 데이터와 출력 데이터 저장 리스트
push_X_data = []
push_y_data = []

# 각 파일을 처리
for file_path in push_file_paths:
    # 파일 읽기
    df = pd.read_csv(file_path, header=None, names=['frame', 'node_id', 'x', 'y', 'z'])
    
    # node_id 별로 데이터 그룹화
    for node_id in df['node_id'].unique():
        node_data = df[df['node_id'] == node_id].set_index('frame')[['x', 'y', 'z']]

        # 슬라이딩 윈도우 적용
        for start in range(0, len(node_data) - window_size):
            end = start + window_size
            # 입력값 (슬라이딩 윈도우)
            window_input = node_data.iloc[start:end].values.astype('float64')

            # 윈도우 별로 정규화
            scaler = RobustScaler()
            scaler_fitted =  scaler.fit(window_input)
            window_input_normalized = scaler_fitted.transform(window_input)  # 각 윈도우별로 정규화
            
            # 출력값 (다음 시점의 (x, y, z) 값)
            next_point = node_data.iloc[end].astype('float64')

            push_X_data.append(window_input_normalized)
            push_y_data.append(next_point.values)
        
# X_data는 입력값 (슬라이딩 윈도우)
# y_data는 출력값 (다음 시점의 (x, y, z))

# numpy 배열로 변환
push_X_data = torch.FloatTensor(np.array(push_X_data))
push_y_data = torch.FloatTensor(np.array(push_y_data))

print("X_data shape:", push_X_data.shape)
print("y_data shape:", push_y_data.shape)

# 결과 예시 (슬라이딩 윈도우의 첫 번째와 두 번째 샘플)
print("Sample input (X):", push_X_data[0])
print("Sample output (y):", push_y_data[0])


X_data shape: torch.Size([78584, 10, 3])
y_data shape: torch.Size([78584, 3])
Sample input (X): tensor([[ 3.0200e-01, -6.6357e-01, -4.6360e-01],
        [ 2.1074e-01, -5.3016e-01,  2.0356e-01],
        [ 1.7761e-01, -4.3395e-01,  5.3460e-01],
        [ 3.3921e-02, -1.1878e-01,  6.0931e-01],
        [ 1.8143e-02,  1.1878e-01, -1.5214e+00],
        [-1.8143e-02,  3.0013e-01, -6.0836e-01],
        [-8.7580e-01,  6.3276e-01, -2.4669e-01],
        [-8.0585e-01,  5.9590e-01, -2.0356e-01],
        [-7.9417e+00,  5.5847e-01,  1.9459e+00],
        [-2.2256e+01, -1.4112e+00,  2.4406e+00]])
Sample output (y): tensor([0.7717, 0.5951, 0.3040])


In [99]:
push_train = TensorDataset(push_X_data,push_y_data)
push_dataloader = DataLoader(push_train, batch_size = 256, shuffle = False)

In [100]:

model = LSTMmodel(3,10).to(device)
def saveModel():
    torch.save(model.state_dict(), f'model/push_lstm_train.pt')
criterion = nn.MSELoss()
optimizer = torch.optim.RAdam(model.parameters(), lr=1e-2)
scheduler = CosineAnnealingLR(optimizer,T_max= 25, eta_min=1e-4)
# 학습 루프
epochs = 100
min_loss = 1e+10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for x, y in push_dataloader:
        input = x.to(device)        
        optimizer.zero_grad()

        # 순전파
        output = model(input)

        # 손실 계산
        loss = criterion(output,y.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    mean_loss =running_loss / len(push_dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(push_dataloader)}")
    if min(mean_loss,min_loss) == mean_loss:
        min_loss = mean_loss
        print('model_saved')
        saveModel()
    

Epoch [1/100], Loss: 0.08972724123648596
model_saved
Epoch [2/100], Loss: 0.052279996843432174
model_saved
Epoch [3/100], Loss: 0.05203852513107298
model_saved
Epoch [4/100], Loss: 0.05172862530252153
model_saved
Epoch [5/100], Loss: 0.051447449420375525
model_saved
Epoch [6/100], Loss: 0.05120818486819438
model_saved
Epoch [7/100], Loss: 0.051067558537367795
model_saved
Epoch [8/100], Loss: 0.05087043172968036
model_saved
Epoch [9/100], Loss: 0.05070180932199431
model_saved
Epoch [10/100], Loss: 0.05057348312160868
model_saved
Epoch [11/100], Loss: 0.050438211982044415
model_saved
Epoch [12/100], Loss: 0.05026689641396075
model_saved
Epoch [13/100], Loss: 0.05007265027648851
model_saved
Epoch [14/100], Loss: 0.04987538544649216
model_saved
Epoch [15/100], Loss: 0.04970048928590862
model_saved
Epoch [16/100], Loss: 0.04955407180171156
model_saved
Epoch [17/100], Loss: 0.04942322425629203
model_saved
Epoch [18/100], Loss: 0.04930761634563293
model_saved
Epoch [19/100], Loss: 0.049198663

In [101]:
import pandas as pd
import glob
import numpy as np
from sklearn.preprocessing import RobustScaler
import torch

# CSV 파일 경로 리스트 (001.csv부터 101.csv까지)
lunge_file_paths = glob.glob('lunge_csv_data/*.csv')  # 'path_to_files/'는 파일들이 저장된 디렉토리 경로입니다.

# 슬라이딩 윈도우 크기
window_size = 10
stride = 1

# 입력 데이터와 출력 데이터 저장 리스트
lunge_X_data = []
lunge_y_data = []

# 각 파일을 처리
for file_path in lunge_file_paths:
    # 파일 읽기
    df = pd.read_csv(file_path, header=None, names=['frame', 'node_id', 'x', 'y', 'z'])
    
    # node_id 별로 데이터 그룹화
    for node_id in df['node_id'].unique():
        node_data = df[df['node_id'] == node_id].set_index('frame')[['x', 'y', 'z']]

        # 슬라이딩 윈도우 적용
        for start in range(0, len(node_data) - window_size):
            end = start + window_size
            # 입력값 (슬라이딩 윈도우)
            window_input = node_data.iloc[start:end].values.astype('float64')

            # 윈도우 별로 정규화
            scaler = RobustScaler()
            scaler_fitted =  scaler.fit(window_input)
            window_input_normalized = scaler_fitted.transform(window_input)  # 각 윈도우별로 정규화
            
            # 출력값 (다음 시점의 (x, y, z) 값)
            next_point = node_data.iloc[end].astype('float64')

            lunge_X_data.append(window_input_normalized)
            lunge_y_data.append(next_point.values)
        
# X_data는 입력값 (슬라이딩 윈도우)
# y_data는 출력값 (다음 시점의 (x, y, z))

# numpy 배열로 변환
lunge_X_data = torch.FloatTensor(np.array(lunge_X_data))
lunge_y_data = torch.FloatTensor(np.array(lunge_y_data))

print("X_data shape:", lunge_X_data.shape)
print("y_data shape:", lunge_y_data.shape)

# 결과 예시 (슬라이딩 윈도우의 첫 번째와 두 번째 샘플)
print("Sample input (X):", lunge_X_data[0])
print("Sample output (y):", lunge_y_data[0])


X_data shape: torch.Size([41094, 10, 3])
y_data shape: torch.Size([41094, 3])
Sample input (X): tensor([[-1.4080, -0.3408, -0.3880],
        [-1.3444, -0.3299, -0.3081],
        [-0.8638, -0.3230, -0.2960],
        [-0.6148, -0.2317, -0.2975],
        [-0.0057, -0.1466, -0.2956],
        [ 0.1462,  0.1466,  0.2956],
        [ 0.0057,  0.4976,  0.5859],
        [ 0.2159,  0.7773,  0.7419],
        [ 0.7244,  0.7673,  0.7526],
        [ 0.8696,  0.9860,  0.7545]])
Sample output (y): tensor([0.5132, 0.5885, 0.2287])


In [102]:
lunge_train = TensorDataset(lunge_X_data,lunge_y_data)
lunge_dataloader = DataLoader(lunge_train, batch_size = 256, shuffle = False)

In [103]:
model = LSTMmodel(3,10).to(device)
def saveModel():
    torch.save(model.state_dict(), f'model/lunge_lstm_train.pt')
criterion = nn.MSELoss()
optimizer = torch.optim.RAdam(model.parameters(), lr=1e-2)
scheduler = CosineAnnealingLR(optimizer,T_max= 25, eta_min=1e-4)
# 학습 루프
epochs = 100
min_loss = 1e+10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for x, y in lunge_dataloader:
        input = x.to(device)        
        optimizer.zero_grad()

        # 순전파
        output = model(input)

        # 손실 계산
        loss = criterion(output,y.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    mean_loss =running_loss / len(lunge_dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(lunge_dataloader)}")
    if min(mean_loss,min_loss) == mean_loss:
        min_loss = mean_loss
        print('model_saved')
        saveModel()

Epoch [1/100], Loss: 0.10454150030146474
model_saved
Epoch [2/100], Loss: 0.05170457745375841
model_saved
Epoch [3/100], Loss: 0.05154399013852481
model_saved
Epoch [4/100], Loss: 0.051334610393976576
model_saved
Epoch [5/100], Loss: 0.05105413235029819
model_saved
Epoch [6/100], Loss: 0.05079250205423891
model_saved
Epoch [7/100], Loss: 0.05061901879051457
model_saved
Epoch [8/100], Loss: 0.05048325744613728
model_saved
Epoch [9/100], Loss: 0.05034857654006955
model_saved
Epoch [10/100], Loss: 0.05020266599513544
model_saved
Epoch [11/100], Loss: 0.05003212915837579
model_saved
Epoch [12/100], Loss: 0.049849505727150425
model_saved
Epoch [13/100], Loss: 0.0496887691226435
model_saved
Epoch [14/100], Loss: 0.04952642458103458
model_saved
Epoch [15/100], Loss: 0.04935422716720134
model_saved
Epoch [16/100], Loss: 0.049197724707979965
model_saved
Epoch [17/100], Loss: 0.04904701296957383
model_saved
Epoch [18/100], Loss: 0.048903993753172594
model_saved
Epoch [19/100], Loss: 0.0487634662